In [1]:
from sympy import Eq, solve, pi
from sympy import lambdify
import numpy as np
from scipy.optimize import fsolve


from sympy import Matrix, cos, sin, symbols

def rotation_matrix_2d(theta):
    """Generate a 2D rotation matrix for a given angle."""
    return Matrix([
        [cos(theta), -sin(theta)],
        [sin(theta), cos(theta)]
    ])

def generate_constraint_equations_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j, u_x_i, u_y_i, u_x_j, u_y_j):
    """Generate the position restraint equations."""
    R_i = Matrix([r_x_i, r_y_i])
    R_j = Matrix([r_x_j, r_y_j])
    local_vector_i = Matrix([u_x_i, u_y_i])
    local_vector_j = Matrix([u_x_j, u_y_j])
    
    transformed_vector_i = rotation_matrix_2d(theta_i) * local_vector_i
    transformed_vector_j = rotation_matrix_2d(theta_j) * local_vector_j

    constraint_equations = R_i + transformed_vector_i - R_j - transformed_vector_j
    return constraint_equations

# def generate_equations_wo_topology_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j):

#     u_x_i, u_y_i, u_x_j, u_y_j = symbols('u_x_i u_y_i u_x_j u_y_j')


#     # Generate the constraint equations
#     equations = generate_constraint_equations_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j, u_x_i, u_y_i, u_x_j, u_y_j)    
    
#     # Return the equation and the symbols
#     #return (equations)
#     return equations

def generate_equations_included_topology_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j, u_x_a, u_y_a, u_x_b, u_y_b):

    u_x_i, u_y_i, u_x_j, u_y_j = symbols('u_x_i u_y_i u_x_j u_y_j')


    # Generate the constraint equations
    equations = generate_constraint_equations_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j, u_x_i, u_y_i, u_x_j, u_y_j)
    
    # Substitute the symbolic variables with the provided values
    equations_substituted = equations.subs({u_x_i: u_x_a, u_y_i: u_y_a, u_x_j: u_x_b, u_y_j: u_y_b})
    
    
    # Return the equation and the symbols
    #return (equations)
    return equations_substituted


In [2]:
def four_bars_mechanism_example_symbolic_full():

    theta_1, theta_2, theta_3, theta_4 = symbols('theta_1 theta_2 theta_3 theta_4')
    r_x_1, r_y_1, r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4 = symbols('r_x_1 r_y_1 r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4')

    L2, L3, L4, Separation = symbols('L2 L3 L4 Separation')



    u_x_1_12 = 0
    u_y_1_12 = 0
    u_x_2_12 = 0
    u_y_2_12 = 0
    restrictions_pr12 = generate_equations_included_topology_pr(theta_1, theta_2, r_x_1, r_y_1, r_x_2, r_y_2, u_x_1_12, u_y_1_12, u_x_2_12, u_y_2_12)

    u_x_2_23 = L2
    u_y_2_23 = 0
    u_x_3_23 = 0
    u_y_3_23 = 0
    restrictions_pr23 = generate_equations_included_topology_pr(theta_2, theta_3, r_x_2, r_y_2, r_x_3, r_y_3, u_x_2_23, u_y_2_23, u_x_3_23, u_y_3_23)

    u_x_3_34 = L3
    u_y_3_34 = 0
    u_x_4_34 = 0
    u_y_4_34 = 0
    restrictions_pr34 = generate_equations_included_topology_pr(theta_3, theta_4, r_x_3, r_y_3, r_x_4, r_y_4, u_x_3_34, u_y_3_34, u_x_4_34, u_y_4_34)

    u_x_1_14 = Separation
    u_y_1_14 = 0
    u_x_4_14 = L4
    u_y_4_14 = 0
    restrictions_pr14 = generate_equations_included_topology_pr(theta_1, theta_4, r_x_1, r_y_1, r_x_4, r_y_4, u_x_1_14, u_y_1_14, u_x_4_14, u_y_4_14)

    # Stacking the matrices vertically to form a single large matrix
    restrictions_to_solve = restrictions_pr12.col_join(restrictions_pr23).col_join(restrictions_pr34).col_join(restrictions_pr14)

    # Substituting the known values into the equations
    restrictions_substituted = restrictions_to_solve.subs({
        r_x_1: 0,
        r_y_1: 0,
        theta_1: 0 })

    # Create a list of equations from the matrix
    equation_list = [Eq(restrictions_substituted[i, 0], 0) for i in range(restrictions_substituted.shape[0])]

    #print(equation_list) 

    return(equation_list)

In [3]:
def four_bars_mechanism_example_symbolic(theta_2_value, L2, L3, L4, Separation):

    theta_1, theta_2, theta_3, theta_4 = symbols('theta_1 theta_2 theta_3 theta_4')
    r_x_1, r_y_1, r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4 = symbols('r_x_1 r_y_1 r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4')


    u_x_1_12 = 0
    u_y_1_12 = 0
    u_x_2_12 = 0
    u_y_2_12 = 0
    restrictions_pr12 = generate_equations_included_topology_pr(theta_1, theta_2, r_x_1, r_y_1, r_x_2, r_y_2, u_x_1_12, u_y_1_12, u_x_2_12, u_y_2_12)

    u_x_2_23 = L2
    u_y_2_23 = 0
    u_x_3_23 = 0
    u_y_3_23 = 0
    restrictions_pr23 = generate_equations_included_topology_pr(theta_2, theta_3, r_x_2, r_y_2, r_x_3, r_y_3, u_x_2_23, u_y_2_23, u_x_3_23, u_y_3_23)

    u_x_3_34 = L3
    u_y_3_34 = 0
    u_x_4_34 = 0
    u_y_4_34 = 0
    restrictions_pr34 = generate_equations_included_topology_pr(theta_3, theta_4, r_x_3, r_y_3, r_x_4, r_y_4, u_x_3_34, u_y_3_34, u_x_4_34, u_y_4_34)

    u_x_1_14 = Separation
    u_y_1_14 = 0
    u_x_4_14 = L4
    u_y_4_14 = 0
    restrictions_pr14 = generate_equations_included_topology_pr(theta_1, theta_4, r_x_1, r_y_1, r_x_4, r_y_4, u_x_1_14, u_y_1_14, u_x_4_14, u_y_4_14)

    # Stacking the matrices vertically to form a single large matrix
    restrictions_to_solve = restrictions_pr12.col_join(restrictions_pr23).col_join(restrictions_pr34).col_join(restrictions_pr14)

    # Substituting the known values into the equations
    restrictions_substituted = restrictions_to_solve.subs({
        r_x_1: 0,
        r_y_1: 0,
        theta_1: 0,
        theta_2: theta_2_value
    })

    # Create a list of equations from the matrix
    equation_list = [Eq(restrictions_substituted[i, 0], 0) for i in range(restrictions_substituted.shape[0])]

    #print(equation_list) 

    return(equation_list)

In [4]:
import plotly.graph_objs as go

def plot_mechanism_updated_json(new_input, Separation):
    # Extract values from the input dictionary using string keys
    r_x_2_value = new_input['r_x_2']
    r_y_2_value = new_input['r_y_2']
    r_x_3_value = new_input['r_x_3']
    r_y_3_value = new_input['r_y_3']
    r_x_4_value = new_input['r_x_4']
    r_y_4_value = new_input['r_y_4']

    print(r_x_2_value)
    
    # Create traces for each line
    trace1 = go.Scatter(x=[r_x_2_value, r_x_3_value], y=[r_y_2_value, r_y_3_value], mode='lines+markers', name='Link 2-3')
    trace2 = go.Scatter(x=[r_x_3_value, r_x_4_value], y=[r_y_3_value, r_y_4_value], mode='lines+markers', name='Link 3-4')
    trace3 = go.Scatter(x=[r_x_4_value, Separation], y=[r_y_4_value, 0], mode='lines+markers', name='Link 4-Base')
    
    # Define layout with equal aspect ratio and specified axis ranges
    layout = go.Layout(title='Four Bars Mechanism Visualization',
                       xaxis=dict(title='X Coordinate', scaleanchor="y", scaleratio=1, range=[-15, 15]),
                       yaxis=dict(title='Y Coordinate', scaleanchor="x", scaleratio=1, range=[-15, 15]),
                       showlegend=True,
                       autosize=False,  # Disable automatic sizing to maintain aspect ratio
                       width=500,  # Set width of the plot
                       height=500)  # Set height of the plot
    
    # Create figure and plot
    fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
    fig.show()

# test_results2 = {
#     'r_x_2': 0.0,
#     'r_x_3': 1.22464679914735e-16,
#     'r_y_2': 0.0,
#     'r_y_3': 2.00000000000000,
#     'r_x_4': 3.06161699786838e-16,
#     'r_y_4': 5.00000000000000
# }

#plot_mechanism_updated_json(test_results2,Separation)
#plot_mechanism_updated_json(new_dict,Separation)


In [19]:

# def convert_floats(input_dict):
#     return {k: float(v) if isinstance(v, sp.Float) else v for k, v in input_dict.items()}


def convert_floats(solution_dict):
    """Convert all values in the dictionary to float."""
    return {key: float(value.evalf()) if isinstance(value, (sp.Number, sp.Symbol)) else value 
            for key, value in solution_dict.items()}


In [21]:
def solve_and_plot_4b(theta_2_range, L2, L3, L4, Separation):

        
    # Solve the equations
    solutions = sp.solve(four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation))

    solutions_theta_3_1 = solutions[0]

    solution_json = {str(key): value for key, value in solutions_theta_3_1.items()}

    solution_json_converted = convert_floats(solution_json)
    plot_mechanism_updated_json(solution_json_converted, Separation)

    

In [22]:
import numpy as np
import plotly.graph_objs as go
import sympy as sp

def plot_mechanism_with_slider(theta_2_range, L2, L3, L4, Separation):
    # Generate solutions for a range of theta_2 values
    solutions_list = []
    for theta_2 in theta_2_range:
        #print(theta_2)
        solutions = sp.solve(four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation))

        #selected_solution = solutions[1]
        # Check if theta_2 is between 0 and pi
        if 0 <= theta_2 <= np.pi:
            selected_solution = solutions[1]
        else:
            selected_solution = solutions[0]
        
        solution_json = {str(key): value for key, value in selected_solution.items()}
        solution_json_converted = convert_floats(solution_json)
        solutions_list.append(solution_json_converted)

    # Generate traces for the plot
    traces = []
    for solution in solutions_list:
        trace1 = go.Scatter(x=[solution['r_x_2'], solution['r_x_3']],
                            y=[solution['r_y_2'], solution['r_y_3']],
                            mode='lines+markers', name='Link 2-3')
        trace2 = go.Scatter(x=[solution['r_x_3'], solution['r_x_4']],
                            y=[solution['r_y_3'], solution['r_y_4']],
                            mode='lines+markers', name='Link 3-4')
        trace3 = go.Scatter(x=[solution['r_x_4'], Separation],
                            y=[solution['r_y_4'], 0],
                            mode='lines+markers', name='Link 4-Base')
        traces.extend([trace1, trace2, trace3])

    # Create the slider steps
    steps = []
    for i, theta_2 in enumerate(theta_2_range):
        step = dict(args=[{"visible": [False] * len(traces)}],  # Set all traces to invisible
                    method="restyle",
                    label=f"{theta_2:.2f}")
        step["args"][0]["visible"][i*3:i*3+3] = [True, True, True]  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(active=10, yanchor="top", steps=steps)]

    # Define layout
    layout = go.Layout(title='Four Bars Mechanism Visualization',
                       xaxis=dict(title='X Coordinate', scaleanchor="y", scaleratio=1, range=[-15, 15]),
                       yaxis=dict(title='Y Coordinate', scaleanchor="x", scaleratio=1, range=[-15, 15]),
                       showlegend=True,
                       autosize=False,
                       width=600,
                       height=600,
                       sliders=sliders)

    fig = go.Figure(data=traces, layout=layout)
    fig.show()

In [59]:
L2, L3, L4, Separation = symbols('L2 L3 L4 Separation')

# # Performing substitution
# substitutions_new = {L2: 2, L3: 2, L4: 2}
# equations_substituted_new = [eq.subs(substitutions_new) for eq in equations]

subs_dict = {L2: 2, L3: 3, L4:5, Separation:4}

[eq.subs(subs_dict) for eq in four_bars_mechanism_example_symbolic_full()]


In [8]:
from sympy import symbols, Eq, cos, sin, diff, Matrix
from sympy import Matrix, Eq


def jacobian():

    #     # Define the symbols
    theta_2, theta_3, theta_4 = symbols('theta_2 theta_3 theta_4')
    r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4 = symbols('r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4')

    # Extract left-hand side of each equation
    exprs = [eq.lhs for eq in four_bars_mechanism_example_symbolic_full()]

    # Add the additional constraint
    exprs.append(theta_2)

    # Convert the list to a Matrix
    f = Matrix(exprs)


    # Variables with respect to which we differentiate
    variables = [r_x_2, r_y_2, theta_2, r_x_3, r_y_3, theta_3, r_x_4, r_y_4, theta_4]

    # Compute the Jacobian
    jacobian_extended = f.jacobian(variables)
    return(jacobian_extended)



In [57]:
jacobian()

Matrix([
[-1,  0,                0,  0,  0,                0,  0,  0,                0],
[ 0, -1,                0,  0,  0,                0,  0,  0,                0],
[ 1,  0, -L2*sin(theta_2), -1,  0,                0,  0,  0,                0],
[ 0,  1,  L2*cos(theta_2),  0, -1,                0,  0,  0,                0],
[ 0,  0,                0,  1,  0, -L3*sin(theta_3), -1,  0,                0],
[ 0,  0,                0,  0,  1,  L3*cos(theta_3),  0, -1,                0],
[ 0,  0,                0,  0,  0,                0, -1,  0,  L4*sin(theta_4)],
[ 0,  0,                0,  0,  0,                0,  0, -1, -L4*cos(theta_4)],
[ 0,  0,                1,  0,  0,                0,  0,  0,                0]])

In [62]:
# from sympy import symbols, Eq, cos, sin, diff, Matrix
# from sympy import Matrix, Eq


# def jacobian_topology(L2_value, L3_value, L4_value, Separation_value):


#     #subs_dict = {L2: 2, L3: 3, L4:5, Separation:4}
#     subs_dict = {L2: L2_value, L3: L3_value, L4:L4_value, Separation:Separation_value}

#         # Define the symbols
#     theta_2, theta_3, theta_4 = symbols('theta_2 theta_3 theta_4')
#     r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4 = symbols('r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4')

#     eq_list_substituted_topology = [eq.subs(subs_dict) for eq in four_bars_mechanism_example_symbolic_full()]

#     print(eq_list_substituted_topology)


#     # Extract left-hand side of each equation
#     exprs = [eq.lhs for eq in eq_list_substituted_topology]

#     # Add the additional constraint
#     exprs.append(theta_2)

#     # Convert the list to a Matrix
#     f = Matrix(exprs)


#     # Variables with respect to which we differentiate
#     variables = [r_x_2, r_y_2, theta_2, r_x_3, r_y_3, theta_3, r_x_4, r_y_4, theta_4]

#     # Compute the Jacobian
#     jacobian_extended = f.jacobian(variables)
#     return(jacobian_extended)



## Position

In [10]:
import sympy as sp

L2 = 2
L3 = 3
L4 = 5
Separation = 4


theta_2 = np.pi/2

In [13]:
sp.solve(four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation))

[{r_x_2: 0.0,
  r_x_3: 1.22464679914735e-16,
  r_y_2: 0.0,
  r_y_3: 2.00000000000000,
  r_y_4: -0.853299832284320,
  r_x_4: -0.926649916142160,
  theta_3: 4.39837028074340,
  theta_4: 0.171499422654302},
 {r_x_2: 0.0,
  r_x_3: 1.22464679914735e-16,
  r_y_2: 0.0,
  r_y_3: 2.00000000000000,
  r_y_4: 4.45329983228432,
  r_x_4: 1.72664991614216,
  theta_3: 0.957519808434573,
  theta_4: -1.09879464065591}]

In [11]:

# Solve the equations
solutions = sp.solve(four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation))

solutions_theta_3_1 = solutions[1]

solution_json = {str(key): value for key, value in solutions_theta_3_1.items()}

In [6]:
solution_json_converted = convert_floats(solution_json)
plot_mechanism_updated_json(solution_json_converted, Separation)

0.0


In [19]:
solve_and_plot_4b(theta_2_range=np.pi/2, L2=3, L3=5, L4=7, Separation=4)

0.0


In [18]:
solve_and_plot_4b(theta_2_range=np.pi/2, L2=3, L3=5, L4=7, Separation=4)

0.0


In [14]:
# Define your theta_2 range and call the function
theta_2_range = np.linspace(np.pi/3, np.pi/2, 20)
plot_mechanism_with_slider(theta_2_range, L2, L3, L4, Separation)


1.0471975511965976
1.074755381491245
1.1023132117858923
1.1298710420805396
1.1574288723751869
1.1849867026698342
1.2125445329644815
1.2401023632591288
1.2676601935537761
1.2952180238484234
1.3227758541430708
1.350333684437718
1.3778915147323654
1.4054493450270127
1.43300717532166
1.4605650056163073
1.4881228359109546
1.515680666205602
1.5432384965002492
1.5707963267948966


In [16]:
# Define your theta_2 range and call the function
theta_2_range = np.linspace(np.pi/4, np.pi/2+np.pi/4, 20)
plot_mechanism_with_slider(theta_2_range, L2, L3, L4, Separation)

0.7853981633974483
0.8680716542813902
0.9507451451653322
1.033418636049274
1.116092126933216
1.198765617817158
1.2814391087011
1.3641125995850416
1.4467860904689838
1.5294595813529255
1.6121330722368676
1.6948065631208094
1.7774800540047513
1.8601535448886932
1.9428270357726352
2.025500526656577
2.1081740175405193
2.190847508424461
2.2735209993084027
2.356194490192345


In [17]:
# Define your theta_2 range and call the function
theta_2_range = np.linspace(np.pi/6, np.pi/2+np.pi/3, 20)
plot_mechanism_with_slider(theta_2_range, L2, L3, L4, Separation)

0.5235987755982988
0.6338300967768881
0.7440614179554772
0.8542927391340664
0.9645240603126557
1.074755381491245
1.1849867026698342
1.2952180238484234
1.4054493450270127
1.515680666205602
1.6259119873841912
1.7361433085627804
1.8463746297413692
1.9566059509199585
2.0668372720985477
2.177068593277137
2.2872999144557262
2.3975312356343155
2.5077625568129047
2.617993877991494


In [24]:
# Define your theta_2 range and call the function
theta_2_range = np.linspace(0, 4*np.pi, 50)
plot_mechanism_with_slider(theta_2_range, L2, L3, L4, Separation)

## Jacobian

In [67]:
four_bars_mechanism_example_symbolic_full()

[Eq(-r_x_2, 0),
 Eq(-r_y_2, 0),
 Eq(L2*cos(theta_2) + r_x_2 - r_x_3, 0),
 Eq(L2*sin(theta_2) + r_y_2 - r_y_3, 0),
 Eq(L3*cos(theta_3) + r_x_3 - r_x_4, 0),
 Eq(L3*sin(theta_3) + r_y_3 - r_y_4, 0),
 Eq(-L4*cos(theta_4) + Separation - r_x_4, 0),
 Eq(-L4*sin(theta_4) - r_y_4, 0)]

In [65]:
jacobian()

Matrix([
[-1,  0,                0,  0,  0,                0,  0,  0,                0],
[ 0, -1,                0,  0,  0,                0,  0,  0,                0],
[ 1,  0, -L2*sin(theta_2), -1,  0,                0,  0,  0,                0],
[ 0,  1,  L2*cos(theta_2),  0, -1,                0,  0,  0,                0],
[ 0,  0,                0,  1,  0, -L3*sin(theta_3), -1,  0,                0],
[ 0,  0,                0,  0,  1,  L3*cos(theta_3),  0, -1,                0],
[ 0,  0,                0,  0,  0,                0, -1,  0,  L4*sin(theta_4)],
[ 0,  0,                0,  0,  0,                0,  0, -1, -L4*cos(theta_4)],
[ 0,  0,                1,  0,  0,                0,  0,  0,                0]])

In [66]:
jacobian().det()

-L3*L4*sin(theta_3)*cos(theta_4) + L3*L4*sin(theta_4)*cos(theta_3)

In [28]:
#jac = jacobian_topology(L2_value=2, L3_value=3, L4_value=5, Separation_value=4)


theta_2, theta_3, theta_4 = symbols('theta_2 theta_3 theta_4')
L2, L3, L4, Separation = symbols('L2 L3 L4 Separation')
jacobian().subs({L2: 2, L3: 2, L4: 2})

# jac.subs({theta_3: 0, theta_4: 0})


# # Performing substitution
# substitutions_new = {L2: 2, L3: 2, L4: 2}
# # equations_substituted_new = [eq.subs(substitutions_new) for eq in equations]

# subs_dict = {L2: 2, L3: 3, L4:5, Separation:4}

# [eq.subs(subs_dict) for eq in four_bars_mechanism_example_symbolic_full()]


Matrix([
[-1,  0,               0,  0,  0,               0,  0,  0,               0],
[ 0, -1,               0,  0,  0,               0,  0,  0,               0],
[ 1,  0, -2*sin(theta_2), -1,  0,               0,  0,  0,               0],
[ 0,  1,  2*cos(theta_2),  0, -1,               0,  0,  0,               0],
[ 0,  0,               0,  1,  0, -2*sin(theta_3), -1,  0,               0],
[ 0,  0,               0,  0,  1,  2*cos(theta_3),  0, -1,               0],
[ 0,  0,               0,  0,  0,               0, -1,  0,  2*sin(theta_4)],
[ 0,  0,               0,  0,  0,               0,  0, -1, -2*cos(theta_4)],
[ 0,  0,               1,  0,  0,               0,  0,  0,               0]])

In [71]:
jacobian_extended = jacobian().subs({L2: 2, L3: 2, L4: 2})


In [86]:
import sympy as sp

L2 = 2
L3 = 3
L4 = 5
Separation = 4


theta_2 = np.pi/2
# Solve the equations
solutions = sp.solve(four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation))

solutions_theta_3_1 = solutions[1]

solution_json = {str(key): value for key, value in solutions_theta_3_1.items()}

solution_json

{'r_x_2': 0.0,
 'r_x_3': 1.22464679914735e-16,
 'r_y_2': 0.0,
 'r_y_3': 2.00000000000000,
 'r_y_4': 4.45329983228432,
 'r_x_4': 1.72664991614216,
 'theta_3': 0.957519808434573,
 'theta_4': -1.09879464065591}

In [89]:
theta3_position = solution_json['theta_3']
theta4_position = solution_json['theta_4']


In [29]:
import sympy as sp

#input topology
L2 = 2
L3 = 3
L4 = 5
Separation = 4

theta_values2 = np.arange(0, 2 * np.pi, np.deg2rad(15))

jac = jacobian().subs({L2: 2, L3: 3, L4: 5})

#solve the position problem (get theta 3 and theta 4 values) and substitute them in the jacobian
for theta_2_val in theta_values2:
    solutions = sp.solve(four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation))

    solutions_theta_3_1 = solutions[1]

    solution_json = {str(key): value for key, value in solutions_theta_3_1.items()}

    theta3_position = solution_json['theta_3']
    theta4_position = solution_json['theta_4']


    #check if the det is invertible
    jacobian_matrix = jac.subs({theta_2: theta_2_val, theta_3: theta3_position, theta_4: theta4_position})

    # Check if the matrix is invertible by computing its determinant
    color = 'green' if jacobian_matrix.det() != 0 else 'red'
            
            # Store the results for visualization
    #theta_combinations.append((theta_2_val, theta_3_val, theta_4_val)) #NOT THE VALS, BUT THE ACTUAL THETA 3 AND 4 POSITION THAT THE MECHANISM IS GOING THROOUGH!!!!!
    theta_combinations.append((theta_2_val, theta3_position, theta4_position))
    colors.append(color)

KeyboardInterrupt: 

In [98]:
theta_combinations

[(0.0, 0.0, 0.0),
 (0.0, 0.0, 0.2617993877991494),
 (0.0, 0.0, 0.5235987755982988),
 (0.0, 0.0, 0.7853981633974483),
 (0.0, 0.0, 1.0471975511965976),
 (0.0, 0.0, 1.308996938995747),
 (0.0, 0.0, 1.5707963267948966),
 (0.0, 0.0, 1.832595714594046),
 (0.0, 0.0, 2.0943951023931953),
 (0.0, 0.0, 2.356194490192345),
 (0.0, 0.0, 2.617993877991494),
 (0.0, 0.0, 2.8797932657906435),
 (0.0, 0.0, 3.141592653589793),
 (0.0, 0.0, 3.4033920413889422),
 (0.0, 0.0, 3.665191429188092),
 (0.0, 0.0, 3.926990816987241),
 (0.0, 0.0, 4.1887902047863905),
 (0.0, 0.0, 4.45058959258554),
 (0.0, 0.0, 4.71238898038469),
 (0.0, 0.0, 4.974188368183839),
 (0.0, 0.0, 5.235987755982988),
 (0.0, 0.0, 5.497787143782138),
 (0.0, 0.0, 5.759586531581287),
 (0.0, 0.0, 6.021385919380436),
 (0.0, 0.2617993877991494, 0.0),
 (0.0, 0.2617993877991494, 0.2617993877991494),
 (0.0, 0.2617993877991494, 0.5235987755982988),
 (0.0, 0.2617993877991494, 0.7853981633974483),
 (0.0, 0.2617993877991494, 1.0471975511965976),
 (0.0, 0.26179

In [25]:
create_3d_scatter(theta_combinations, colors)


NameError: name 'create_3d_scatter' is not defined

In [82]:
# Define the range for theta_2, theta_3, and theta_4 in 15 degree steps
theta_values = np.arange(0, 2 * np.pi, np.deg2rad(15))

theta_values2 = np.arange(0, np.deg2rad(15), np.deg2rad(15))

# Lists to store the results for visualization
theta_combinations = []
colors = []

# Iterate over the theta values
for theta_2_val in theta_values:
    print(theta_2_val)
    for theta_3_val in theta_values:
        for theta_4_val in theta_values:
            # Get the Jacobian matrix for the current combination of theta values
            jacobian_matrix = jacobian_extended.subs({theta_2: theta_2_val, theta_3: theta_3_val, theta_4: theta_4_val})
            
            # Check if the matrix is invertible by computing its determinant
            color = 'green' if jacobian_matrix.det() != 0 else 'red'
            
            # Store the results for visualization
            theta_combinations.append((theta_2_val, theta_3_val, theta_4_val))
            colors.append(color)

len(theta_combinations), len(colors)

In [73]:
# Define the range for theta_2, theta_3, and theta_4 in 15 degree steps
theta_values = np.arange(0, 2 * np.pi, np.deg2rad(15))

theta_values2 = np.arange(0, np.pi/2, np.deg2rad(15))

# Lists to store the results for visualization
theta_combinations = []
colors = []

# Iterate over the theta values
for theta_2_val in theta_values:
    print(theta_2_val)
    for theta_3_val in theta_values:
        for theta_4_val in theta_values:
            # Get the Jacobian matrix for the current combination of theta values
            jacobian_matrix = jacobian_extended.subs({theta_2: theta_2_val, theta_3: theta_3_val, theta_4: theta_4_val})
            
            # Check if the matrix is invertible by computing its determinant
            color = 'green' if jacobian_matrix.det() != 0 else 'red'
            
            # Store the results for visualization
            theta_combinations.append((theta_2_val, theta_3_val, theta_4_val))
            colors.append(color)

len(theta_combinations), len(colors)

0.0
0.2617993877991494
0.5235987755982988
0.7853981633974483
1.0471975511965976
1.308996938995747
1.5707963267948966
1.832595714594046
2.0943951023931953
2.356194490192345
2.617993877991494
2.8797932657906435
3.141592653589793
3.4033920413889422
3.665191429188092
3.926990816987241
4.1887902047863905
4.45058959258554
4.71238898038469
4.974188368183839
5.235987755982988
5.497787143782138
5.759586531581287
6.021385919380436


(13824, 13824)

In [75]:
import plotly.graph_objects as go
import numpy as np

def create_3d_scatter(theta_points, colors):
    """Create a 3D scatter plot using Plotly."""
    fig = go.Figure(data=[go.Scatter3d(
        x=[point[0] for point in theta_points],
        y=[point[1] for point in theta_points],
        z=[point[2] for point in theta_points],
        mode='markers',
        marker=dict(
            size=6,
            color=colors,                # set color to an array/list of desired values
            opacity=0.8
        )
    )])

    fig.update_layout(scene=dict(
            xaxis_title='Theta 2',
            yaxis_title='Theta 3',
            zaxis_title='Theta 4'
        ),
        width=700,
        margin=dict(r=20, b=10, l=10, t=10))
    fig.show()

In [76]:
create_3d_scatter(theta_combinations, colors)


# theta_combinations = []
# colors = []